In [7]:
import numpy as np
import pickle
import time
from operator import add
%load_ext line_profiler

In [8]:
# board = [[1x1 pieces], [1x2 pieces], [2x1 pieces], [2x2 pieces]] (the lower-left block of each piece)
start_board_state = [[[0, 0], [1, 1], [2, 1], [3, 0]], [[0, 3], [0, 1], [3, 3], [3, 1]], [[1, 2]], [[1, 3]]]

In [9]:
class Node:
    def __init__(self, board_state):
        self.parent = None
        self.board_state = board_state
        self.children = []
        self.add_to_history(self.compress(self.board_state))
    
    def add_to_history(self, compressed_board):    
        global history
        history.add(compressed_board)      
        return
    
    def compress(self, board_state):       
        output = []
        for piece_type in board_state:
            output.append(sorted(piece_type))
        return str(output)
    
    def  generate_children(self):
        global history
        for piece_type in range(len(self.board_state)):
            for piece in range(len(self.board_state[piece_type])):
                for direction in [[-1, 0], [0, -1], [0, 1], [1, 0]]:
                    potential_child = self.move_if_possible(piece_type, piece, direction)
                    if potential_child == None:
                        pass
                    elif self.compress(potential_child) not in history:
                        self.children.append(Node(potential_child))
        return
    
    def move_if_possible(self, piece_type, piece, direction):
#         expanded_board = [[self.expand_piece(i, j) for j in self.board_state[i]] for i in range(len(self.board_state))]
#         expanded_piece = expanded_board[piece_type][piece]
#         moved_piece = [list(map(add, piece, direction)) for piece in expanded_piece]
#         del expanded_board[piece_type][piece]
#         for block in moved_piece:
#             if block[0] < 0 or block[0] > 3 or block[1] < 0 or block[1] > 4:
#                 return
#             for piece_type_wo in expanded_board:
#                 for piece_wo in piece_type_wo:
#                     if block in piece_wo:
#                         return
#         output = [[piece[:] for piece in piece_type] for piece_type in self.board_state]
#         output[piece_type][piece] = moved_piece[0]
#         return output
        board_without_piece = [[list(piece) for piece in piece_type] for piece_type in self.board_state]
        del board_without_piece[piece_type][piece]
        moved_piece = [sum(x) for x in zip(self.board_state[piece_type][piece][:], direction)]
        expanded_piece = self.expand_piece(piece_type, moved_piece)
        for block in expanded_piece:
            if block[0] < 0 or block[0] > 3 or block[1] < 0 or block[1] > 4:
                return
            for piece_type_wo in board_without_piece:
                for piece_wo in piece_type_wo:
                    if block == piece_wo:
                        return
        all_blocks_wo = []
        upper_right_expanded = max(expanded_piece)
        for piece_type_wo in range(len(board_without_piece)):
            for piece_wo in board_without_piece[piece_type_wo]:
                if piece_wo[0] <= upper_right_expanded[0] or piece_wo[1] <= upper_right_expanded[1]:
                    expanded_wo = self.expand_piece(piece_type_wo, piece_wo)
                    for block in expanded_wo:
                        if block in expanded_piece:
                            return
        output = [[piece[:] for piece in piece_type] for piece_type in self.board_state]
        output[piece_type][piece] = moved_piece
        return output
    
    def expand_piece(self, piece_type, piece):
        if piece_type == 0:
            output = [piece]
        elif piece_type == 1:
            output = [piece, list(map(add, piece, [0, 1]))]
        elif piece_type == 2:
            output = [piece, list(map(add, piece, [1, 0]))]
        elif piece_type == 3:
            output = [piece, list(map(add, piece, [1, 0])), 
                      list(map(add, piece, [0, 1])), 
                      list(map(add, piece, [1, 1]))]
        return output

    def draw_board(self):
        expanded_board = []
        for piece_type in range(len(self.board_state)):
            for piece in self.board_state[piece_type]:
                expanded_board.append(self.expand_piece(piece_type, piece))
        board = np.full((5, 4), ' ')
        for i in range(10):
            # X-Wert
            for x in range(len(expanded_board[i])):
                board[expanded_board[i][x][1]][expanded_board[i][x][0]] = i
        return np.flip(np.fliplr(board))

In [10]:
def parent_pointer(child, parent):
    child.parent = parent

In [11]:
def get_layer(root, depth):
    root = [root]
    for layer in range(depth):
        new_root = []
        for node in root:    
            for child in node.children:
                new_root.append(child)
            root = new_root
    return root

In [12]:
def get_last_layer(root):
    root = [root]
    depth = 0
    while True:
        new_root = []
        for node in root:    
            for child in node.children:
                new_root.append(child)
        if new_root == []:
            return root
        root = new_root
        depth += 1

In [13]:
class Tree:
    def __init__(self, root):
        self.root = root
        self.history = []

In [14]:
def save(root):
    with open(file, 'wb') as f:
        pickle.dump(root, f, pickle.HIGHEST_PROTOCOL)
        
def load(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [15]:
%%prun
history = {*()}

root = Node(start_board_state)
i = 0
layer = get_layer(root, 0)
while not layer == []:
    [node.generate_children() for node in layer]
    for node in layer:
        for child in node.children:
            parent_pointer(node, child)
    print('layer:', i, 'nodes:', len(layer))
    i += 1
    layer = get_layer(root, i)

layer: 0 nodes: 1
layer: 1 nodes: 4
layer: 2 nodes: 10
layer: 3 nodes: 13
layer: 4 nodes: 18
layer: 5 nodes: 16
layer: 6 nodes: 19
layer: 7 nodes: 20
layer: 8 nodes: 22
layer: 9 nodes: 32
layer: 10 nodes: 38
layer: 11 nodes: 42
layer: 12 nodes: 36
layer: 13 nodes: 34
layer: 14 nodes: 42
layer: 15 nodes: 40
layer: 16 nodes: 34
layer: 17 nodes: 30
layer: 18 nodes: 29
layer: 19 nodes: 36
layer: 20 nodes: 47
layer: 21 nodes: 60
layer: 22 nodes: 54
layer: 23 nodes: 48
layer: 24 nodes: 48
layer: 25 nodes: 39
layer: 26 nodes: 35
layer: 27 nodes: 45
layer: 28 nodes: 52
layer: 29 nodes: 68
layer: 30 nodes: 74
layer: 31 nodes: 69
layer: 32 nodes: 51
layer: 33 nodes: 50
layer: 34 nodes: 65
layer: 35 nodes: 77
layer: 36 nodes: 100
layer: 37 nodes: 132
layer: 38 nodes: 145
layer: 39 nodes: 172
layer: 40 nodes: 201
layer: 41 nodes: 238
layer: 42 nodes: 282
layer: 43 nodes: 342
layer: 44 nodes: 376
layer: 45 nodes: 396
layer: 46 nodes: 426
layer: 47 nodes: 404
layer: 48 nodes: 403
layer: 49 nodes: 44

In [16]:
i = 0
total_nodes = 0
while True:
    layer = get_layer(root, i)
    if layer == []:
        break
    total_nodes += len(layer)
    i += 1
print('total number of possible board states: %s'% total_nodes)

total number of possible board states: 25955


https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html

https://towardsdatascience.com/speed-up-jupyter-notebooks-20716cbe2025